In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model
# model = YOLO('yolov8n.pt')          ### Pre-trained weights

model = YOLO('runs/detect/train4/weights/best.pt')          ### weights from trained model

# Open the video file
video_path = "Test_Video.mp4"



results = model(video_path)
ball_predictions = []
ball_positions = []
 
for result in results:
    helper = result.boxes.cls
    index = (helper == 0).nonzero()
    print(helper)
    print(index)
    print(result.boxes.xyxy[index])

print("Pruend rresults")




WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/487) c:\Users\Sadiq\Documents\School\ComputerVision\project\Test_Video.mp4: 384x640 14 Players, 1 Referee, 78.9ms
video 1/1 (frame 2/487) c:\Users\Sadiq\Documents\School\ComputerVision\project\Test_Video.mp4: 384x640 14 Players, 1 Referee, 43.0ms
video 1/1 (frame 3/487) c:\Users\Sadiq\Documents\School\ComputerVision\project\Test_Video.mp4: 384x640 14 Players, 1 Referee, 44.0ms
video 1/1 (frame 4/487) c:\Users\Sadiq\Documents\School\Com

In [3]:
#Find ball classes to get position in each frame

for result in results:
    classes = result.boxes.cls
    ball_index = (classes == 0).nonzero()

    ball_position = result.boxes.xyxy[ball_index]
    ball_positions.append(ball_position)


In [4]:

cap = cv2.VideoCapture('test_video.mp4')

# Parameters for the mass-spring-damper system
mass = 8.0
damping = 1.0
stiffness = 0.2

frame_count = 0
velocity = np.array([0.0, 0.0])

# Initialize the last_known_position to the center of the frame
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))


# Calculate the width and height of the bounding box
w, h = frame_width // 2, frame_height // 2

last_known_position = np.array([w, h], dtype=float)
# print(f'w:{w}, h:{h}')

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # If the tensor for this frame is not empty, draw the bounding box and zoom into it
        if ball_positions[frame_count].numel() > 0:
            x1, y1, x2, y2 = map(int, ball_positions[frame_count][0][0].numpy())

            # Calculate the center of the ball
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            cv2.circle(frame, (cx, cy), radius=22, color=(255, 0, 255), thickness=6)

            # Calculate the new bounding box ensuring it's within the frame boundaries
            x1, y1 = max(0, int(cx - w // 2)), max(0, int(cy - h // 2))
            x2, y2 = min(frame_width, int(cx + w // 2)), min(frame_height, int(cy + h // 2))
            target_position = np.array([cx, cy])
            
            # Update the position based on the mass-spring-damper system
            # Calculate the center of the zoomed frame
            zoomed_frame_center = last_known_position + np.array([w // 2, h // 2])

            # Calculate the displacement from the zoomed frame center to the target position
            displacement = target_position - zoomed_frame_center
            acceleration = (stiffness * displacement - damping * velocity) / mass
            velocity += acceleration
            # Ensure velocity is in the direction of displacement
            velocity = np.sign(displacement) * np.abs(velocity)
            #print(f"Dispalcement:{displacement}")
        else:
            # If the ball is not detected, apply a damping to the velocity
            velocity -= 0.3 * velocity / mass
        
        last_known_position += velocity
        #print(f"Velocity: {velocity} ")
        # Ensure last_known_position stays within the frame boundaries
        last_known_position[0] = np.clip(last_known_position[0], 0, frame_width - w)
        last_known_position[1] = np.clip(last_known_position[1], 0, frame_height - h)

        x1, y1 = map(int, last_known_position)
        x2, y2 = x1 + w, y1 + h
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        #print([x1,y1,x2,y2])
        #print(f'frame center:{x2-x1 //2},{y2-y1//2}')
        # Create a zoomed in frame
        zoomed_frame = frame[y1:y2, x1:x2]
        zoomed_frame = cv2.resize(zoomed_frame, (frame_width, frame_height))
        
        #print(f'Frame center:{x1+x2 //2, y1+y2 //2}    Velocity:{velocity}    Displacement{displacement}')
        cv2.imshow('Zoomed Frame', zoomed_frame)
        cv2.imshow('Frame', frame)
        # print(f'Center of zoomed frame:{y2+y1 //2}, {x2+x1//2}, Ball center:{cy, cx}')
        # Press Q on keyboard to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

    frame_count += 1

cap.release()
cv2.destroyAllWindows()